In [1]:
import torch
from utils import print_gpu_memory
print_gpu_memory()
import os
import sys
base_path = r"C:\Users\KAI\Coding\ThinkOnward_challenge\thinkOnward_TSClassification"
data_path = r"\data\building-instinct-starter-notebook\Starter notebook"
sys.path.append(base_path+data_path)
sys.path.append(base_path+"\kai")
import pandas as pd
from tqdm import tqdm
from utils import (calculate_average_hourly_energy_consumption, train_model, get_pred, calculate_hierarchical_f1_score,
sample_submission_generator)
from preprocessing import Preprocessor
# file_path = base_path + data_path + r"\building-instinct-train-data"
# df_features = Preprocessor.load_standard_df(file_path)
# df_features.sort_index(inplace=True)
# df_features.to_parquet(base_path + '/preprocessed_data/data_9000.parquet', engine='pyarrow')

# df_loaded_1 = pd.read_parquet(base_path + '/preprocessed_data/data_3000.parquet', engine='pyarrow')
# df_loaded_2 = pd.read_parquet(base_path + '/preprocessed_data/data_6000.parquet', engine='pyarrow')
# df_loaded_3 = pd.read_parquet(base_path + '/preprocessed_data/data_9000.parquet', engine='pyarrow')

# df_features = pd.concat([df_loaded_1, df_loaded_2, df_loaded_3], axis=0)
# df_features.sort_index(inplace=True)
# df_features.to_parquet(base_path + '/preprocessed_data/standard_data.parquet', engine='pyarrow')

df_features = pd.read_parquet(base_path + '/preprocessed_data/standard_data.parquet', engine='pyarrow')
df_features.sort_index(inplace=True)

Allocated memory: 0.00 GB
Cached memory: 0.00 GB
Total memory: 12.00 GB
Unused memory: 12.00 GB


In [25]:
from preprocessing import TargetPreprocessor
load_filepath_labels = os.path.join(base_path + data_path,'building-instinct-train-label', 'train_label.parquet')#path to the train label file
df_targets = pd.read_parquet(load_filepath_labels, engine='pyarrow')
df_targets.sort_index(inplace=True)

df_targets_res = df_targets[df_targets.building_stock_type == "residential"].filter(like='_res').copy()
df_targets_com = df_targets[df_targets.building_stock_type == "commercial"].filter(like='_com').copy()
target_preprocessor = TargetPreprocessor()
df_targets_res, association_dict_res, encoder_res = target_preprocessor.preprocess_res(df_targets_res)
df_targets_com, association_dict_com, encoder_com = target_preprocessor.preprocess_com(df_targets_com)

In [3]:
common_indices = df_features.index.intersection(df_targets_com.index)
# Filter the data
X_com = df_features[df_features.index.isin(common_indices)]
X_com = torch.tensor(X_com.values, dtype=torch.float32)
y_com = df_targets_com[df_targets_com.index.isin(common_indices)]
y_com = torch.tensor(y_com.values, dtype=torch.float32)

X_res = df_features[df_features.index.isin(df_targets_res.index)]
X_res = torch.tensor(X_res.values, dtype=torch.float32)
y_res = df_targets_res[df_targets_res.index.isin(df_features.index)]
y_res = torch.tensor(y_res.values, dtype=torch.float32)

## classifier

In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt

X, y = df_features, df_targets["building_stock_type"].map({"residential": 0, "commercial": 1})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = Pipeline([('preprocessor', ColumnTransformer([
            ('scaler', StandardScaler(), X.columns),
            ('encoder', OneHotEncoder(), [])
        ])),
        ('classifier', RandomForestClassifier(random_state=42))
    ])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f1_score(y_test, y_pred, average='macro'))
print(confusion_matrix(y_test, y_pred))

0.992323346090601


In [31]:
df_test = pd.read_parquet(base_path + '/preprocessed_data/data_test.parquet', engine='pyarrow')
df_test.sort_index(inplace=True)

clf = Pipeline([('preprocessor', ColumnTransformer([
            ('scaler', StandardScaler(), X.columns),
            ('encoder', OneHotEncoder(), [])
        ])),
        ('classifier', RandomForestClassifier(random_state=42))
    ])
clf.fit(X, y)
y_pred = clf.predict(df_test)
df_test["building_stock_type"] = y_pred

## com model

In [4]:
# from torch.utils.data import DataLoader, Dataset
# from models import MultiTaskLSTM, CustomLoss, TimeSeriesDataset
# from datetime import datetime

# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# torch.cuda.empty_cache()

# # model parameters
# batch_size = 16
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# input_size, sequence_length = 1, X_com.shape[1]
# num_classes_categorical = y_com.shape[1]
# hidden_size = 64
# num_epochs = 20

# weight_numerical = 10e-4
# weight_categorical = 1.0

# best_combined_loss = float('inf')
# checkpoint_filename = base_path+ f'/kai/checkpoints/model_checkpoint_{datetime.now().strftime("%m_%d_%H_%M")}.pth.tar'

# # create dataloaders
# dataloader_com = DataLoader(TimeSeriesDataset(X_com, y_com), batch_size=batch_size, shuffle=True)
# dataloader_res = DataLoader(TimeSeriesDataset(X_res, y_res), batch_size=batch_size, shuffle=True)

# model = MultiTaskLSTM(input_size, hidden_size, num_classes_categorical)
# model = model.to(device)
# criterion = CustomLoss(association_dict_com)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# for epoch in range(num_epochs):
#     model.train()
#     epoch_loss = 0.0
#     for X_batch, y_categorical_batch in dataloader_com:
#         X_batch = X_batch.view(X_batch.shape[0], sequence_length, input_size)
#         X_batch = X_batch.to(device)
#         y_categorical_batch = y_categorical_batch.to(device)
#         optimizer.zero_grad()
#         categorical_pred = model(X_batch)
#         loss, loss_numerical, loss_categorical = criterion(categorical_pred, y_categorical_batch)
        
#         # Combine losses with adjusted weights
#         combined_loss = weight_numerical * loss_numerical + weight_categorical * loss_categorical

#         combined_loss.backward()
#         optimizer.step()
#         epoch_loss += loss.item()
#     print(f'Epoch [{epoch+1}/{num_epochs}], '
#                   f'Total Loss: {loss.item():.4f}, '
#                   f'Numerical Loss: {loss_numerical.item():.4f}, '
#                   f'Categorical Loss: {loss_categorical.item():.4f}, '
#                   f'Combined Loss: {combined_loss.item():.4f}, '
#                   f'weights: {weight_numerical:.4f}, {weight_categorical:.4f}')
    
#     if combined_loss.item() < best_combined_loss:
#         torch.save({'epoch': epoch + 1,
#                     'model_state_dict': model.state_dict(),
#                     'optimizer_state_dict': optimizer.state_dict(),
#                     'loss': combined_loss.item(),}, checkpoint_filename)


In [5]:
from torch.utils.data import DataLoader, Dataset
from models import MultiTaskLSTM, CustomLoss, TimeSeriesDataset
from datetime import datetime

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
batch_size, input_size, sequence_length, hidden_size, num_classes_categorical = 16, 1, X_com.shape[1], 64, y_com.shape[1]
checkpoint_filename = base_path+ f'/kai/checkpoints/com_model_checkpoint_08_06_14_49.pth.tar'

dataloader_com = DataLoader(TimeSeriesDataset(X_com, y_com), batch_size=batch_size, shuffle=True)
model = MultiTaskLSTM(input_size, hidden_size, num_classes_categorical)
model = model.to(device)

checkpoint = torch.load(checkpoint_filename)
model.load_state_dict(checkpoint['model_state_dict'])
for X_batch, y_categorical_batch in dataloader_com:
    for i in [0, 1]:
        _X_batch = X_batch[:, :].view(X_batch.shape[0], sequence_length, input_size)
        _X_batch = _X_batch.to(device)
        categorical_pred = model.predict(_X_batch, association_dict_com)
        print(categorical_pred)
        print(y_categorical_batch[i, :])
        break
    break

C:\Users\KAI\AppData\Local\Temp\ipykernel_14864\3095246467.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_filename)


tensor([[2.2376e+00, 1.9897e+03, 3.5269e+02, 3.4938e+02, 8.7197e+00, 1.8646e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00],
        [2.2499e+00, 1.9759e+03, 3.5024e+02, 3.4684e+02, 8.7135e+00, 1.8552e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00],
        [2.2634e+00, 1.9178e+03, 3.3992e+02, 3.3631e+02, 8.6316e+00, 1.8104e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+

### create the original com df

In [6]:
from postprocessing import inverse_process
arr_df = inverse_process(categorical_pred, encoder_com) # todo: standardize the target numerical columns in preprocessing
display(arr_df)

,in.number_of_stories_com,in.vintage_com,in.tstat_clg_sp_f..f_com,in.tstat_htg_sp_f..f_com,in.weekday_opening_time..hr_com,in.weekday_operating_hours..hr_com,in.comstock_building_type_group_com,in.heating_fuel_com,in.hvac_category_com,in.ownership_type_com,in.wall_construction_type_com
0,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
1,2,1970 to 1979,80,72,8.75,18.5,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
2,2,Before 1946,80,72,8.75,18.0,Office,Electricity,Small Packaged Unit,leased,SteelFramed
3,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
4,2,1980 to 1989,80,72,8.75,18.5,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
5,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
6,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
7,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
8,2,Before 1946,80,72,8.75,18.25,Office,Electricity,Small Packaged Unit,leased,SteelFramed
9,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed


In [7]:
# file_path = base_path + data_path + r"\building-instinct-test-data"
# df_features = Preprocessor.load_standard_df(file_path)
# df_features.sort_index(inplace=True)
# df_features.to_parquet(base_path + '/preprocessed_data/data_test.parquet', engine='pyarrow')

df_test = pd.read_parquet(base_path + '/preprocessed_data/data_test.parquet', engine='pyarrow')
df_test.sort_index(inplace=True)

# TODO: clf that predicts com or res
# then filter by com & res

df_test = df_test[df_test.index.isin(df_features.index)]
X_test_com = torch.tensor(df_test.values, dtype=torch.float32)
# X_test_com = X_test_com.to(device)

In [21]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
def free_gpu_memory():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
free_gpu_memory()
model = MultiTaskLSTM(input_size, hidden_size, num_classes_categorical)
model = model.to(device)
criterion = CustomLoss(association_dict_com)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
checkpoint_filename = base_path+ f'/kai/checkpoints/com_model_checkpoint_08_06_14_49.pth.tar'
checkpoint = torch.load(checkpoint_filename)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
best_loss = checkpoint['loss']

predictions = []
dataloader = DataLoader(TimeSeriesDataset(X_test_com, torch.zeros(X_test_com.shape[0], y_com.shape[1])), batch_size=16, shuffle=False)

print_gpu_memory()
for X_batch, y_categorical_batch in tqdm(dataloader):
    model.eval()
    _X_batch = X_batch.view(X_batch.shape[0], sequence_length, input_size)
    _X_batch = _X_batch.to(device)
    with torch.no_grad():
        categorical_pred = model.predict(_X_batch, association_dict_com)
    predictions.append(categorical_pred.cpu())
    del _X_batch, categorical_pred
    free_gpu_memory()
predictions = torch.cat(predictions, dim=0)
arr_df = inverse_process(predictions, encoder_com)

C:\Users\KAI\AppData\Local\Temp\ipykernel_14864\934529602.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_filename)


Allocated memory: 0.02 GB
Cached memory: 10.16 GB
Total memory: 12.00 GB
Unused memory: 11.98 GB


100%|██████████| 90/90 [00:24<00:00,  3.62it/s]


In [24]:
for col in arr_df.columns:
    print(arr_df[col].unique())

[2]
['Before 1946' '1980 to 1989' '1960 to 1969' '1970 to 1979' '1946 to 1959']
[80]
[72]
[8.5 8.75]
[18.0 18.75 18.5 18.25]
['Office' 'Mercantile']
['Electricity']
['Small Packaged Unit']
['leased']
['SteelFramed']
